In [1]:
import pandas as pd
import syft as sy
import numpy as np

sy.logger.remove()

### Logging into the domain Nodes

In [2]:
# We will login into Canada and Italy domain node
ca_domain_node = sy.login(email="sheldon@caltech.edu", password="bazinga", port=8081)
it_domain_node = sy.login(email="sheldon@caltech.edu", password="bazinga", port=8082)

Connecting to http://localhost:8081... done! 	 Logging into canada... done!
Connecting to http://localhost:8082... done! 	 Logging into istat... done!


In [3]:
ca_domain_node.datasets

,description,tags,name,data,id
0,A collection of reports from Canada's statisti...,[],Canada Trade Data - First 40000 rows,"[{'name': '40k-feb2020-numpy-Tensor', 'id': '9...",4e5f8211-5175-4be8-8eb1-9ac8f72cc41a


In [4]:
it_domain_node.datasets

,tags,id,description,data,name
0,[],15037af1-28df-4fe1-b399-342219f26caa,A collection of reports from Italy's statistic...,"[{'name': 'Italy-Numpy-feb2020-Tensor', 'id': ...",Italy Trade Data - First 40000 rows


In [7]:
# Select the dataset pointers using the index

ca_dataset_ptr = ca_domain_node.datasets[0]['']
it_dataset_ptr = it_domain_node.store[0]

In [8]:
# Verfiy the dataset tensors via tags

ca_dataset_ptr.tags, it_dataset_ptr.tags

(['#40k-feb2020-numpy-Tensor'], ['#Italy-Numpy-feb2020-Tensor'])

In [9]:
from syft.core.tensor.smpc.mpc_tensor import MPCTensor

In [10]:
parties = [ca_domain_node, it_domain_node]

In [39]:
# Converting ca_dataset_ptr to mpc tensor

mpc_tensor = MPCTensor(secret=ca_dataset_ptr, shape=(40000, 3), parties=parties)

In [40]:
ptr = mpc_tensor.child[0]

In [41]:
ptr2 = mpc_tensor.child[1]

In [42]:
ptr.request("Asdf")

In [43]:
ptr2.request("asdf")

In [45]:
ptr.get()

In [38]:
ptr2.get()

ShareTensor(child=[[ 4532506801847576509 -5789304215717750947  1163686722992199311]
 [-4422254520734781819 -8249451413581095868 -3966820270973885030]
 [ 4795987604492745870 -2581759301704949106 -1491814546804702664]
 ...
 [ 4637192718173863100  6387477136743811706  7600121608769756504]
 [-4351128119122932103 -7043690909825214826  1259354745974367362]
 [ 6034584541746107832  4416082405803516230 -6228474764170114642]])

In [25]:
# Performing a Private / Public Addition

# Creating an array of ones of size (40000, 3)
# Converting float to int, for SMPC to work
e = np.ones((40000, 3))
e = e.astype(int)

result = mpc_tensor + e

In [27]:
result.child[0].request("asdf")

In [28]:
result.child[1].request("asdf2")

In [30]:
result.child[0].get()

ShareTensor(child=[[ 8359510376939655160  -788730512376427001  4951942160648009674]
 [ -836833501850931068 -2309934752468721704  1475883799364072075]
 [ 4937883618576837590  8938681542697651924   936752248871503693]
 ...
 [ -901695604009217114  5959742822944013405 -4965474910037576997]
 [  325710419047006759  8192880480563781754  9173897704867404370]
 [ 6871693176828493880 -3609639827667495353 -3008895202542392416]])

In [31]:
result.child[1].get()

ShareTensor(child=[[-8359510376939655158   788730512376427492 -4951942160648000388]
 [  836833501850931070  2309934752468722523 -1475883799363955470]
 [-4937883618576837588 -8938681542697651097  -936752248870008517]
 ...
 [  901695604009217116 -5959742822944012806  4965474910037577278]
 [ -325710419047006757 -8192880480563781313 -9173897704867401382]
 [-6871693176828493878  3609639827667495510  3008895202542395280]])